In [7]:
#imports!
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from google.colab import files

# User csv upload and processing
uploaded = files.upload()
file_name = list(uploaded.keys())[0]
data = pd.read_csv(file_name)

# Custom descaling functions to Descale Barretts scaling function
def descale_lat(scaled_lat):
    return (scaled_lat * 10) + 14

def descale_lon(scaled_lon):
    return (scaled_lon * 10) + 120

# Descaling the lat and longs
data['AvgLat'] = descale_lat(data['AvgLat'])
data['AvgLon'] = descale_lon(data['AvgLon'])

# Normalizing the data and isolating features and target
scaler = MinMaxScaler()
data[['AvgLat', 'AvgLon', 'AvgHead']] = scaler.fit_transform(data[['AvgLat', 'AvgLon', 'AvgHead']])
features = ['AvgLat', 'AvgLon', 'AvgHead']
target = ['AvgLat', 'AvgLon', 'AvgHead']

# Reshaping inputs to be [samples, time steps, features]
X = np.array(data[features]).reshape((data.shape[0], 1, 3))
y = np.array(data[target])

# Creating the LSTM model with initial input layer, 2 LSTM layers, and 1 Dense layer with 3 outputs (AvgLat, AvgLon, AvgHead)
model = Sequential()
model.add(LSTM(128, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(LSTM(64))
model.add(Dense(3))  # Predicting 3 values: AvgLat, AvgLon, AvgHead

#Compiling and training data from source
model.compile(optimizer='adam', loss='mean_squared_error', metrics=[tf.keras.metrics.MeanAbsoluteError()])
history = model.fit(X, y, epochs=100, validation_split=0.2, verbose=1)

#predicting values
y_pred = model.predict(X)

# Setting up loss functions (MSE AND MAE) - Kinda overkill :0
mse = mean_squared_error(y, y_pred)
mae = mean_absolute_error(y, y_pred)
print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Error: {mae}')

# Inversing the predicted values
y_pred_inverse = scaler.inverse_transform(y_pred)

# Descaling the predicted values
y_pred_inverse[:, 0] = descale_lat(y_pred_inverse[:, 0])
y_pred_inverse[:, 1] = descale_lon(y_pred_inverse[:, 1])

# Invering the actual values for the user to compare to predicted values
y_actual_inverse = scaler.inverse_transform(y)
y_actual_inverse[:, 0] = descale_lat(y_actual_inverse[:, 0])
y_actual_inverse[:, 1] = descale_lon(y_actual_inverse[:, 1])

# mapping predictions back
data['Pred_AvgLat'] = y_pred_inverse[:, 0]
data['Pred_AvgLon'] = y_pred_inverse[:, 1]
data['Pred_AvgHead'] = y_pred_inverse[:, 2]
data['Actual_AvgLat'] = y_actual_inverse[:, 0]
data['Actual_AvgLon'] = y_actual_inverse[:, 1]
data['Actual_AvgHead'] = y_actual_inverse[:, 2]

#Saving and Downloading CSV
output_df = data[['Hour', 'Cluster', 'Actual_AvgLat', 'Actual_AvgLon', 'Actual_AvgHead', 'Pred_AvgLat', 'Pred_AvgLon', 'Pred_AvgHead']].copy()
output_file_name = 'predictions_output.csv'
output_df.to_csv(output_file_name, index=False)
files.download(output_file_name)

Saving clusterdata_centroids.csv to clusterdata_centroids (5).csv
Epoch 1/100
779/779 [==============================] - 12s 10ms/step - loss: 0.0174 - mean_absolute_error: 0.0489 - val_loss: 1.4224e-04 - val_mean_absolute_error: 0.0082
Epoch 2/100
779/779 [==============================] - 6s 7ms/step - loss: 4.5616e-05 - mean_absolute_error: 0.0039 - val_loss: 6.3074e-05 - val_mean_absolute_error: 0.0047
Epoch 3/100
779/779 [==============================] - 8s 10ms/step - loss: 2.7675e-05 - mean_absolute_error: 0.0030 - val_loss: 5.0075e-05 - val_mean_absolute_error: 0.0043
Epoch 4/100
779/779 [==============================] - 6s 8ms/step - loss: 2.0211e-05 - mean_absolute_error: 0.0027 - val_loss: 4.3365e-05 - val_mean_absolute_error: 0.0045
Epoch 5/100
779/779 [==============================] - 7s 9ms/step - loss: 1.3652e-05 - mean_absolute_error: 0.0022 - val_loss: 2.4232e-05 - val_mean_absolute_error: 0.0027
Epoch 6/100
779/779 [==============================] - 6s 7ms/step - l

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>